# **Homework 3 - Convolutional Neural Network**

This is the example code of homework 3 of the machine learning course by Prof. Hung-yi Lee.

In this homework, you are required to build a convolutional neural network for image classification, possibly with some advanced training tips.


There are three levels here:

**Easy**: Build a simple convolutional neural network as the baseline. (2 pts)

**Medium**: Design a better architecture or adopt different data augmentations to improve the performance. (2 pts)

**Hard**: Utilize provided unlabeled data to obtain better results. (2 pts)

## **About the Dataset**

The dataset used here is food-11, a collection of food images in 11 classes.

For the requirement in the homework, TAs slightly modified the data.
Please DO NOT access the original fully-labeled training data or testing labels.

Also, the modified dataset is for this course only, and any further distribution or commercial use is forbidden.

In [ ]:
# Download the dataset
# You may choose where to download the data.

# Google Drive
!gdown --id '1awF7pZ9Dz7X1jn1_QAiKN-_v56veCEKy' --output food-11.zip

# Dropbox
# !wget https://www.dropbox.com/s/m9q6273jl3djall/food-11.zip -O food-11.zip

# MEGA
# !sudo apt install megatools
# !megadl "https://mega.nz/#!zt1TTIhK!ZuMbg5ZjGWzWX1I6nEUbfjMZgCmAgeqJlwDkqdIryfg"

# Unzip the dataset.
# This may take some time.
!unzip -q food-11.zip

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1awF7pZ9Dz7X1jn1_QAiKN-_v56veCEKy
From (redirected): https://drive.google.com/uc?id=1awF7pZ9Dz7X1jn1_QAiKN-_v56veCEKy&confirm=t&uuid=598e4f44-ddd4-4e74-8e23-dc1155129d6a
To: /content/food-11.zip
 79% 763M/963M [00:08<00:10, 19.6MB/s]Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/urllib3/response.py", line 779, in _error_catcher
    yield
  File "/usr/local/lib/python3.11/dist-packages/urllib3/response.py", line 904, in _raw_read
    data = self._fp_read(amt, read1=read1) if not fp_closed else b""
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/urllib3/response.py", line 887, in _fp_read
    return self._fp.read(amt) if amt is not

## **Import Packages**

First, we need to import packages that will be used later.

In this homework, we highly rely on **torchvision**, a library of PyTorch.

In [ ]:
# Import necessary packages.
import numpy as np
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image
# "ConcatDataset" and "Subset" are possibly useful when doing semi-supervised learning.
from torch.utils.data import ConcatDataset, DataLoader, Subset
from torchvision.datasets import DatasetFolder

# This is for the progress bar.
from tqdm.auto import tqdm

## **Dataset, Data Loader, and Transforms**

Torchvision provides lots of useful utilities for image preprocessing, data wrapping as well as data augmentation.

Here, since our data are stored in folders by class labels, we can directly apply **torchvision.datasets.DatasetFolder** for wrapping data without much effort.

Please refer to [PyTorch official website](https://pytorch.org/vision/stable/transforms.html) for details about different transforms.

In [ ]:
# It is important to do data augmentation in training.
# However, not every augmentation is useful.
# Please think about what kind of augmentation is helpful for food recognition.
train_tfm = transforms.Compose([
    # 随机裁剪并缩放到128x128，增强尺度不变性
    transforms.RandomResizedCrop((128, 128), scale=(0.8, 1.0), ratio=(0.9, 1.1)),
    # 随机水平翻转，模拟左右视角
    transforms.RandomHorizontalFlip(),
    # 随机旋转，增强旋转鲁棒性
    transforms.RandomRotation(degrees=15),
    # 随机调整亮度、对比度、饱和度
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    # 转为Tensor
    transforms.ToTensor(),
    # 随机擦除部分区域，防止模型过度依赖局部特征
    transforms.RandomErasing(p=0.2, scale=(0.02, 0.2), ratio=(0.3, 3.3)),

])

# We don't need augmentations in testing and validation.
# All we need here is to resize the PIL image and transform it into Tensor.
test_tfm = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

In [ ]:
# Batch size for training, validation, and testing.
# A greater batch size usually gives a more stable gradient.
# But the GPU memory is limited, so please adjust it carefully.
batch_size = 128

# Construct datasets.
# The argument "loader" tells how torchvision reads the data.
train_set = DatasetFolder("food-11/training/labeled", loader=lambda x: Image.open(x), extensions="jpg", transform=train_tfm)
valid_set = DatasetFolder("food-11/validation", loader=lambda x: Image.open(x), extensions="jpg", transform=test_tfm)
unlabeled_set = DatasetFolder("food-11/training/unlabeled", loader=lambda x: Image.open(x), extensions="jpg", transform=train_tfm)
test_set = DatasetFolder("food-11/testing", loader=lambda x: Image.open(x), extensions="jpg", transform=test_tfm)

# Construct data loaders.
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=True, num_workers=8, pin_memory=True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

## **Model**

The basic model here is simply a stack of convolutional layers followed by some fully-connected layers.

Since there are three channels for a color image (RGB), the input channels of the network must be three.
In each convolutional layer, typically the channels of inputs grow, while the height and width shrink (or remain unchanged, according to some hyperparameters like stride and padding).

Before fed into fully-connected layers, the feature map must be flattened into a single one-dimensional vector (for each image).
These features are then transformed by the fully-connected layers, and finally, we obtain the "logits" for each class.

### **WARNING -- You Must Know**
You are free to modify the model architecture here for further improvement.
However, if you want to use some well-known architectures such as ResNet50, please make sure **NOT** to load the pre-trained weights.
Using such pre-trained models is considered cheating and therefore you will be punished.
Similarly, it is your responsibility to make sure no pre-trained weights are used if you use **torch.hub** to load any modules.

For example, if you use ResNet-18 as your model:

model = torchvision.models.resnet18(pretrained=**False**) → This is fine.

model = torchvision.models.resnet18(pretrained=**True**)  → This is **NOT** allowed.

In [ ]:
import torchvision.models as models
import torch.nn as nn

class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        # 使用ResNet18作为特征提取器，不加载预训练权重
        self.backbone = models.resnet18(pretrained=True)
        # 修改输入层以适应3通道输入（默认就是3，无需更改）
        # 修改输出层为11类
        num_features = self.backbone.fc.in_features
        self.backbone.fc = nn.Linear(num_features, 11)

    def forward(self, x):
        # 输入x: [batch_size, 3, 128, 128]
        # 输出: [batch_size, 11]
        x = self.backbone(x)
        return x

## **Training**

You can finish supervised learning by simply running the provided code without any modification.

The function "get_pseudo_labels" is used for semi-supervised learning.
It is expected to get better performance if you use unlabeled data for semi-supervised learning.
However, you have to implement the function on your own and need to adjust several hyperparameters manually.

For more details about semi-supervised learning, please refer to [Prof. Lee's slides](https://speech.ee.ntu.edu.tw/~tlkagk/courses/ML_2016/Lecture/semi%20(v3).pdf).

Again, please notice that utilizing external data (or pre-trained model) for training is **prohibited**.

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Subset
from tqdm import tqdm # 导入tqdm用于显示进度条

# 假设 batch_size 已在其他地方定义，例如：
# batch_size = 64

class PseudoLabelDataset(Dataset):
    """
    一个自定义的PyTorch Dataset，用于封装带有伪标签的数据子集。
    它将原始数据集的特定子集与为这些子集生成的伪标签关联起来。
    """
    def __init__(self, subset: Subset, pseudo_labels: list):
        """
        初始化伪标签数据集。

        Args:
            subset (torch.utils.data.Subset): 原始数据集的一个子集，其中包含被选为生成伪标签的样本。
                                              这个子集包含了图像数据，但它的原始标签将被忽略。
            pseudo_labels (list or torch.Tensor): 与subset中每个样本对应的伪标签列表或张量。
                                                 顺序必须与subset中的样本顺序一致。
        """
        self.subset = subset  # 存储原始数据集的子集
        self.pseudo_labels = pseudo_labels  # 存储为该子集生成的伪标签

    def __getitem__(self, idx: int):
        """
        根据索引获取一个数据样本及其对应的伪标签。

        Args:
            idx (int): 样本在当前PseudoLabelDataset中的索引。

        Returns:
            tuple: (img, pseudo_label)
                   img (torch.Tensor): 图像数据。
                   pseudo_label (int): 对应的伪标签。
        """
        # 从原始子集中获取图像和原始标签。
        # 注意：我们只关心图像数据，原始标签（_）在这里被忽略，因为我们将使用伪标签。
        img, _ = self.subset[idx]
        return img, self.pseudo_labels[idx]

    def __len__(self) -> int:
        """
        返回数据集中样本的总数。
        """
        return len(self.subset)


def get_pseudo_labels(dataset: Dataset, model: nn.Module, threshold: float = 0.65) -> PseudoLabelDataset:
    """
    根据模型预测为无标签数据生成伪标签。
    这个函数会遍历整个数据集，使用当前模型进行预测，
    并选择那些预测置信度高于给定阈值的样本作为伪标签样本。

    Args:
        dataset (torch.utils.data.Dataset): 待生成伪标签的数据集（通常是无标签或未使用的有标签数据）。
        model (torch.nn.Module): 用于生成预测的模型。在生成伪标签时，模型应处于评估模式。
        threshold (float, optional): 置信度阈值。只有当模型对某个预测的最高概率超过此阈值时，
                                     该预测才会被接受为伪标签。默认为0.65。

    Returns:
        PseudoLabelDataset: 一个新的Dataset，其中包含被选中的图像及其对应的伪标签。
                            如果没有任何样本的置信度达到阈值，则返回一个空的Subset。
    """
    # 确定设备（GPU或CPU）
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # 创建一个数据加载器，用于遍历整个数据集。
    # shuffle=False 确保数据按原始顺序处理，以便后续索引匹配。
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)

    # 将模型设置为评估模式。
    # 这会关闭Dropout和BatchNorm等层的训练行为，确保预测结果的稳定性。
    model.eval()

    # 定义Softmax层，用于将模型的logits（原始输出）转换为概率分布。
    softmax = nn.Softmax(dim=-1)

    # 存储被选中的样本在原始数据集中的索引
    selected_indices = []
    # 存储被选中的样本对应的伪标签
    selected_labels = []

    # 用于跟踪当前批次在整个数据集中的起始索引
    idx_base = 0

    # 遍历数据加载器中的所有批次，并显示进度条
    for batch in tqdm(data_loader, desc="Generating Pseudo Labels"):
        imgs, _ = batch # 获取图像数据，忽略原始标签（如果有的话）
        batch_size_now = imgs.size(0) # 获取当前批次的实际大小
        imgs = imgs.to(device) # 将图像数据移动到指定设备

        # 在不计算梯度的上下文下进行推理，以节省内存和计算。
        with torch.no_grad():
            logits = model(imgs) # 模型前向传播，得到原始输出（logits）

        probs = softmax(logits) # 将logits转换为概率分布

        # 找到每个样本预测概率最高的类别及其对应的概率值
        max_probs, pseudo_labels = torch.max(probs, dim=1) # max_probs是最高概率，pseudo_labels是对应的类别索引

        # 创建一个布尔掩码，标记哪些样本的最高概率超过了设定的阈值
        mask = max_probs > threshold

        # 遍历当前批次中的每个样本
        for i in range(batch_size_now):
            if mask[i]: # 如果当前样本的置信度高于阈值
                # 记录该样本在原始数据集中的全局索引
                selected_indices.append(idx_base + i)
                # 记录该样本的伪标签（将其从GPU移回CPU并转换为Python标量）
                selected_labels.append(pseudo_labels[i].cpu().item())

        # 更新下一个批次的起始索引
        idx_base += batch_size_now

    # 如果没有样本的置信度达到阈值，则返回一个空的Subset
    if len(selected_indices) == 0:
        # 使用Subset包装空列表，确保返回类型一致
        return Subset(dataset, [])

    # 使用Subset根据selected_indices从原始数据集中提取子集
    subset = Subset(dataset, selected_indices)

    # 创建并返回一个PseudoLabelDataset实例，其中包含选定的图像和它们的伪标签
    pseudo_dataset = PseudoLabelDataset(subset, selected_labels)

    # 将模型重新设置为训练模式，以便后续的训练过程
    model.train()

    return pseudo_dataset



In [ ]:
# "cuda" only when GPUs are available.
device = "cuda" if torch.cuda.is_available() else "cpu"

# Initialize a model, and put it on the device specified.
model = Classifier().to(device)
model.device = device

# For the classification task, we use cross-entropy as the measurement of performance.
criterion = nn.CrossEntropyLoss()

# Initialize optimizer, you may fine-tune some hyperparameters such as learning rate on your own.
optimizer = torch.optim.Adam(model.parameters(), lr=0.0003, weight_decay=1e-5)

# The number of training epochs.
n_epochs = 80

# Whether to do semi-supervised learning.
do_semi = True

for epoch in range(n_epochs):
    # ---------- TODO ----------
    # In each epoch, relabel the unlabeled dataset for semi-supervised learning.
    # Then you can combine the labeled dataset and pseudo-labeled dataset for the training.
    # if do_semi:
    #     # Obtain pseudo-labels for unlabeled data using trained model.
    #     pseudo_set = get_pseudo_labels(unlabeled_set, model)

    #     # Construct a new dataset and a data loader for training.
    #     # This is used in semi-supervised learning only.
    #     concat_dataset = ConcatDataset([train_set, pseudo_set])
    #     train_loader = DataLoader(concat_dataset, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
    if do_semi:
        pseudo_set = get_pseudo_labels(unlabeled_set, model, threshold=0.65)
        concat_dataset = ConcatDataset([train_set, pseudo_set])
        train_loader = DataLoader(concat_dataset, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)

    # ---------- Training ----------
    # Make sure the model is in train mode before training.
    model.train()

    # These are used to record information in training.
    train_loss = []
    train_accs = []

    # Iterate the training set by batches.
    for batch in tqdm(train_loader):

        # A batch consists of image data and corresponding labels.
        imgs, labels = batch

        # Forward the data. (Make sure data and model are on the same device.)
        logits = model(imgs.to(device))

        # Calculate the cross-entropy loss.
        # We don't need to apply softmax before computing cross-entropy as it is done automatically.
        loss = criterion(logits, labels.to(device))

        # Gradients stored in the parameters in the previous step should be cleared out first.
        optimizer.zero_grad()

        # Compute the gradients for parameters.
        loss.backward()

        # Clip the gradient norms for stable training.
        grad_norm = nn.utils.clip_grad_norm_(model.parameters(), max_norm=10)

        # Update the parameters with computed gradients.
        optimizer.step()

        # Compute the accuracy for current batch.
        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

        # Record the loss and accuracy.
        train_loss.append(loss.item())
        train_accs.append(acc)

    # The average loss and accuracy of the training set is the average of the recorded values.
    train_loss = sum(train_loss) / len(train_loss)
    train_acc = sum(train_accs) / len(train_accs)

    # Print the information.
    print(f"[ Train | {epoch + 1:03d}/{n_epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")

    # ---------- Validation ----------
    # Make sure the model is in eval mode so that some modules like dropout are disabled and work normally.
    model.eval()

    # These are used to record information in validation.
    valid_loss = []
    valid_accs = []

    # Iterate the validation set by batches.
    for batch in tqdm(valid_loader):

        # A batch consists of image data and corresponding labels.
        imgs, labels = batch

        # We don't need gradient in validation.
        # Using torch.no_grad() accelerates the forward process.
        with torch.no_grad():
          logits = model(imgs.to(device))

        # We can still compute the loss (but not the gradient).
        loss = criterion(logits, labels.to(device))

        # Compute the accuracy for current batch.
        acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()

        # Record the loss and accuracy.
        valid_loss.append(loss.item())
        valid_accs.append(acc)

    # The average loss and accuracy for entire validation set is the average of the recorded values.
    valid_loss = sum(valid_loss) / len(valid_loss)
    valid_acc = sum(valid_accs) / len(valid_accs)

    # Print the information.
    print(f"[ Valid | {epoch + 1:03d}/{n_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")

## **Testing**

For inference, we need to make sure the model is in eval mode, and the order of the dataset should not be shuffled ("shuffle=False" in test_loader).

Last but not least, don't forget to save the predictions into a single CSV file.
The format of CSV file should follow the rules mentioned in the slides.

### **WARNING -- Keep in Mind**

Cheating includes but not limited to:
1.   using testing labels,
2.   submitting results to previous Kaggle competitions,
3.   sharing predictions with others,
4.   copying codes from any creatures on Earth,
5.   asking other people to do it for you.

Any violations bring you punishments from getting a discount on the final grade to failing the course.

It is your responsibility to check whether your code violates the rules.
When citing codes from the Internet, you should know what these codes exactly do.
You will **NOT** be tolerated if you break the rule and claim you don't know what these codes do.


In [ ]:
# Make sure the model is in eval mode.
# Some modules like Dropout or BatchNorm affect if the model is in training mode.
model.eval()

# Initialize a list to store the predictions.
predictions = []

# Iterate the testing set by batches.
for batch in tqdm(test_loader):
    # A batch consists of image data and corresponding labels.
    # But here the variable "labels" is useless since we do not have the ground-truth.
    # If printing out the labels, you will find that it is always 0.
    # This is because the wrapper (DatasetFolder) returns images and labels for each batch,
    # so we have to create fake labels to make it work normally.
    imgs, labels = batch

    # We don't need gradient in testing, and we don't even have labels to compute loss.
    # Using torch.no_grad() accelerates the forward process.
    with torch.no_grad():
        logits = model(imgs.to(device))

    # Take the class with greatest logit as prediction and record it.
    predictions.extend(logits.argmax(dim=-1).cpu().numpy().tolist())

In [ ]:
# Save predictions into the file.
with open("predict.csv", "w") as f:

    # The first row must be "Id, Category"
    f.write("Id,Category\n")

    # For the rest of the rows, each image id corresponds to a predicted class.
    for i, pred in  enumerate(predictions):
         f.write(f"{i},{pred}\n")